In [13]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"]
                               )
school_data_complete.head()


#District Summary
total_schools = len(school_data_complete['school_name'].unique())
total_students = school_data_complete['Student ID'].count()
total_budget = school_data['budget'].sum()
avg_math_score = school_data_complete['math_score'].mean()
avg_reading_score = school_data_complete['reading_score'].mean()
passing_rate = (avg_math_score + avg_reading_score) / 2
passing_math_count = len(school_data_complete[school_data_complete[
    'math_score'] > 69].groupby('Student ID')['math_score'].count())
passing_math_percentage = (passing_math_count / total_students) * 100
passing_reading_count = len(school_data_complete[school_data_complete[
    'reading_score'] > 69].groupby('Student ID')['reading_score'].count())
passing_reading_percentage = (passing_reading_count / total_students) * 100

#District Data Frame
district_summary = pd.DataFrame({'total districts': total_schools, 'total students': total_students,
                                'total budget': total_budget, 'avg math score': avg_math_score,
                                'avg reading score' : avg_reading_score, 'passing rate' : passing_rate,
                                'passing math percentage' : passing_math_percentage, 
                                'passing reading percentage' : passing_reading_percentage}, 
                                index = [0])

#School Summary
school_names = school_data_complete.groupby([school_data_complete['school_name']]).mean()

school_type = school_data.set_index(['school_name'])['type']

students_per_school = school_data_complete["school_name"].value_counts()

school_budget = school_data_complete.groupby(['school_name']).mean()['budget']

per_student_budget = school_budget / total_students 

avg_math = school_data_complete["math_score"].mean()
students_passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70,:]
passing_math = len(students_passing_math)
pct_passing_math = (passing_math / total_students)*100   
   
avg_reading = school_data_complete["reading_score"].mean()
students_passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70,:]
passing_reading = len(students_passing_reading)
pct_passing_reading = (passing_reading / total_students)*100

overall_passing_rate = (pct_passing_math + pct_passing_reading) / 2

#School Data Frame
school_summary = pd.DataFrame({'school type': school_type,
                                'students per school': students_per_school,
                                'school budget': school_budget,
                                '$ per Student' : per_student_budget, 
                                'Avg Math Score' : avg_math, 'Avg Reading Score' : avg_reading,
                                '% Passing Math' : pct_passing_math, '% Passing Reading' : pct_passing_reading,
                                'Overall Passing' : overall_passing_rate})

#Top 5 schools
top_5 = school_summary.sort_values(['Overall Passing'], ascending = False).head(5)
top_5


#Bottom 5 Schools
bottom_5 = school_summary.sort_values(['Overall Passing'], ascending = False).tail(5)
bottom_5


#Scores by Grade
avg_math_by_grade = round(school_data_complete.groupby(["grade"]).mean()["math_score"])
avg_math_by_grade

avg_reading_by_grade = round(school_data_complete.groupby(["grade"]).mean()["reading_score"])
avg_reading_by_grade


# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary["Spending Ranges (Per Student)"] = pd.cut(per_student_budget,spending_bins,labels = group_names)
school_summary

#Scores by Spending
avg_math_score_spending = school_summary.groupby(
    ["Spending Ranges (Per Student)"]).mean()["Avg Math Score"]
avg_reading_score_spending = school_summary.groupby(
    ["Spending Ranges (Per Student)"]).mean()["Avg Reading Score"]
students_passing_math_spending = school_summary.groupby(
    ["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
students_passing_reading_spending = school_summary.groupby(
    ["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_pass_rate_spending = (students_passing_math_spending + students_passing_reading_spending)/2

scores_school_spending = pd.DataFrame({"Avg Math Score By Spending":avg_math_score_spending,
                                     "Avg Reading Score By Spending":avg_reading_score_spending,
                                     "% Passing Math By Spending":students_passing_math_spending,
                                     "%Passing Reading By Spending":students_passing_reading_spending ,
                                     "Overall Passing rate By Spending":overall_pass_rate_spending})

# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary["Size of the school"] = pd.cut(school_summary[
    "students per school"], size_bins, labels = group_names)

#Scores by School Size
avg_math_score_size = school_summary.groupby(
    ["Size of the school"]).mean()["Avg Math Score"]
avg_reading_score_size = school_summary.groupby(
    ["Size of the school"]).mean()["Avg Reading Score"]
percentage_passing_math_size = school_summary.groupby(
    ["Size of the school"]).mean()["% Passing Math"]
percentage_passing_reading_size = school_summary.groupby(
    ["Size of the school"]).mean()["% Passing Reading"]
overall_passing_rate = school_summary.groupby(
    ["Size of the school"]).mean()["Overall Passing"]


scores_school_size = pd.DataFrame({"Average Math Score":avg_math_score_size,
                                  "Average Reading Score":avg_reading_score_size,
                                  "% Passing Math":percentage_passing_math_size,
                                  "% Passing Reading":percentage_passing_reading_size,
                                  "Overall Passing rate":overall_passing_rate})
scores_school_size = scores_school_size[["Average Math Score","Average Reading Score",
                                        "% Passing Math","% Passing Reading",]]

#Scores by School Type
avg_math_score_type = school_summary.groupby(
    ["school type"]).mean()["Avg Math Score"]
avg_reading_score_type = school_summary.groupby(
    ["school type"]).mean()["Avg Reading Score"]
percentage_passing_math_type = school_summary.groupby(
    ["school type"]).mean()["% Passing Math"]
percentage_passing_reading_type = school_summary.groupby(
    ["school type"]).mean()["% Passing Reading"]
overall_passing_rate_type = school_summary.groupby(
    ["school type"]).mean()["Overall Passing"]


scores_school_type = pd.DataFrame({"Average Math Score":avg_math_score_type,
                                  "Average Reading Score":avg_reading_score_type,
                                  "% Passing Math":percentage_passing_math_type,
                                  "% Passing Reading":percentage_passing_reading_type,
                                  "Overall Passing rate":overall_passing_rate_type})
scores_school_type = scores_school_type[["Average Math Score","Average Reading Score",
                                        "% Passing Math","% Passing Reading","Overall Passing rate"]]
scores_school_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing rate
school type,,,,,
Charter,78.985371,81.87784,74.980853,85.805463,80.393158
District,78.985371,81.87784,74.980853,85.805463,80.393158
